In [1]:
import os
os.chdir('../iml-hack')
os.curdir

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

import data_prep
import utils

from typing import List, Tuple

In [2]:
si = SimpleImputer(strategy='mean')
knni = KNNImputer(n_neighbors=100)

In [3]:
data_x, data_y, data_perform = data_prep.load_train(perform=True, group=False)
print(np.isnan(data_x).sum())
data_x = knni.fit_transform(data_x)
print(np.isnan(data_x).sum())
print(data_x.shape, data_y.shape, data_perform.shape)
data_x = StandardScaler().fit_transform(data_x)

28933
0
(8000, 116) (8000,) (8000,)


In [4]:
class NetRegClass(nn.Module):
    def __init__(self, input_size: int, hidden_size: List[int], num_classes: int = 3, num_reg: int = 1, dropout: float = 0.1):
        super(NetRegClass, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            # nn.BatchNorm1d(hidden_size[0]),
            # nn.SiLU(),
            nn.ReLU(),
        )
        self.ff = nn.ModuleList(
        nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.BatchNorm1d(out_dim),
            # nn.SiLU(),
            nn.ReLU(),
            nn.Dropout(dropout)
        ) for in_dim, out_dim in zip(hidden_size[:-1], hidden_size[1:])
        )
        self.classification_head = nn.Linear(hidden_size[-1], num_classes)
        self.regression_head = nn.Linear(hidden_size[-1], num_reg)
    
    def forward(self, x):
        x = self.embedding(x)
        for layer in self.ff:
            x = x + layer(x)
        return self.classification_head(x), self.regression_head(x)

In [5]:
folds = data_prep.prep_crossvalidation(data_x, data_y, data_perform, batch_size=64, n_splits=5, seed=42)
train_dl, val_dataset = folds[0]

In [6]:
nrc = NetRegClass(116, [100, 100, 100], dropout=0.3)

utils.train_with_performance(
    model=nrc,
    dataloader=train_dl,
    criterion=utils.cm_loss,
    optimizer=optim.Adam(nrc.parameters(), lr=0.00001, weight_decay=0.0001),
    num_epochs=100,
    eval_every=20,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    val_dataset=val_dataset,
    alpha=0.,
    beta=0.1,
    gamma=0.001,
)

Epoch 19: Validation metric: 1.0137


KeyboardInterrupt: 